## Imports

In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
from shapely import wkt

## Ingest Data

In [9]:
df = pd.read_csv("../data/parking_data_2020.csv")

/var/folders/lr/597pxhmx2z78r4vysc_dvsth0000gn/T/ipykernel_34096/3730071359.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../data/parking_data_2020.csv", parse_dates=True)


In [12]:
df['Citation Issued DateTime'] = pd.to_datetime(df['Citation Issued DateTime'])

In [13]:
df.head()

,Citation Number,Citation Issued DateTime,Violation,Violation Description,Citation Location,Vehicle Plate State,Vehicle Plate,Fine Amount,Date Added,geom,Neighborhoods,SF Find Neighborhoods,Current Police Districts,Current Supervisor Districts,Analysis Neighborhoods
0,PD34805304,2020-01-01 00:03:00,V22500H,DBL PARK,PALM AVE,CA,8KD4235,110.0,05/11/2021 12:00:00 AM,POINT (-122.45789733699996 37.783540370000026),11.0,11.0,8.0,6.0,31.0
1,926909395,2020-01-01 00:15:00,TRC7.2.41,PK PHB OTD,735 TERRY A FRANCOIS BOULEVARD,CA,8LCZ875,106.0,03/14/2021 12:00:00 AM,POINT (-122.38609376 37.767347795000035),34.0,34.0,1.0,10.0,4.0
2,PD34751430,2020-01-01 00:15:00,PK6.01G,NOPRK 10P6,2100 SUNNYDALE AVE,CA,5GXH820,23.0,04/30/2020 12:00:00 AM,POINT (-122.41881402299998 37.71817499200006),73.0,73.0,9.0,9.0,19.0
3,926604011,2020-01-01 00:15:00,TRC7.2.41,PK PHB OTD,621 TERRY A FRANCOIS BLVD,CA,8CKN522,106.0,07/27/2020 12:00:00 AM,POINT (-122.38627475799996 37.769162866000045),34.0,34.0,1.0,10.0,4.0
4,926909406,2020-01-01 00:20:00,TRC7.2.41,PK PHB OTD,730 TERRY A FRANCOIS BOULEVARD,CA,7NWC642,106.0,07/27/2020 12:00:00 AM,POINT (-122.38621425199995 37.76739727200004),34.0,34.0,1.0,10.0,4.0


In [15]:
# top 50 most common violations in SF

violation_gb = df.groupby(["Violation", "Violation Description"]).size().sort_values(ascending=False)
violation_gb.head(50)

Violation    Violation Description
TRC7.2.22    STR CLEAN                159434
TRC7.2.23B   MTR OUT DT                35678
TRC7.2.20    RES/OT                    16447
TRC7.2.23A   METER DTN                 12417
V22500H      DBL PARK                   9605
TRC7.2.26    YEL ZONE                   8476
V22500E      DRIVEWAY                   8037
V5204A       REG TABS                   7967
TRC7.2.40    PRK PROHIB                 7290
TRC7.2.41    PK PHB OTD                 6633
TRC7.2.101A  FAIL DISPL                 6565
TRC7.2.35    PRK GRADE                  6123
V5200        NO PLATES                  5335
V22500F      ON SIDEWLK                 5307
TRC7.2.83    TRK ZONE                   4981
TRC7.2.38    PK STANDS                  4434
TRC7.2.25    RED ZONE                   4282
V21211       BLK BIKE L                 2890
TRC7.2.27    WHITE ZONE                 2091
TRC7.2.70    OBSTRCT TF                 1755
V22500I      BUS ZONE                   1739
V22502A      OVR 18 

In [16]:
# get geometry

df.loc[:, 'geom'] = df.geom.apply(wkt.loads)

In [18]:
# get latitude and longitude information from points

df.loc[:, 'long'] = df.geom.apply(lambda point: point.x)
df.loc[:, 'lat'] = df.geom.apply(lambda point: point.y)

In [ ]:
# plot heatmap of parking incidents

# fig = px.density_mapbox(df_sample[~df['SF Find Neighborhoods'].isnull()], lat='long', lon='lat', z='count', 
#                         mapbox_style="stamen-terrain")
 
# fig